#Installing dependencies

In [ ]:
!pip install -qU langchain langchain-openai langchain-community langchain-core langgraph chromadb langchain-tavily langchain-groq

In [ ]:
!pip -q install docx2txt pypdf unstructured dotenv

#Setting structures

In [ ]:
from langchain_core.messages import BaseMessage
from typing import List, Literal
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

class RouteDecision(BaseModel):
    route: Literal["rag", "answer", "end"]
    reply: str | None = Field(None, description="Filled only when route == 'end'")

class RagJudge(BaseModel):
    sufficient: bool

class AgentState(TypedDict, total=False):
    messages: List[BaseMessage]
    route:    Literal["rag", "answer", "end"]
    rag:      str
    web:      str

class State(TypedDict):
    """State for our chatbot - this holds the conversation history"""
    # The add_messages function handles appending new messages to the conversation
    messages: Annotated[list[BaseMessage], add_messages]

#Nodes and edges

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

router_llm = ChatGroq(model="llama3-70b-8192", temperature=0)\
             .with_structured_output(RouteDecision)

def from_router(st: AgentState) -> Literal["rag", "answer", "end"]:
    return st["route"]

def after_rag(st: AgentState) -> Literal["answer", "web"]:
    return st["route"]

def after_web(_) -> Literal["answer"]:
    return "answer"

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage
def rag_node(state: AgentState) -> AgentState:
    print("RAG node invoked")
    query = next((m.content for m in reversed(state["messages"])
                  if isinstance(m, HumanMessage)), "")

    chunks = rag_search_tool.invoke({"query": query})
    # Use structured output to judge if RAG results are sufficient
    judge_messages = [
        ("system", (
            "You are a judge evaluating if the retrieved information is sufficient "
            "to answer the user's question. Consider both relevance and completeness."
        )),
        ("user", f"Question: {query}\n\nRetrieved info: {chunks}\n\nIs this sufficient to answer the question?")
    ]
    verdict: RagJudge = judge_llm.invoke(judge_messages)

    return {
        **state,
        "rag": chunks,
        "route": "answer" if verdict.sufficient else "web"
    }

def web_node(state: AgentState) -> AgentState:
    print("Web node invoked")
    query = next((m.content for m in reversed(state["messages"])
                  if isinstance(m, HumanMessage)), "")
    snippets = web_search_tool.invoke({"query": query})
    return {**state, "web": snippets, "route": "answer"}

def chatbot_node(state: AgentState) -> AgentState:
    print("Chatbot node invoked")
    """
    The chatbot node that decides whether to use tools or provide direct responses
    """
    messages = state.get("messages", [])

    # Add system prompt
    system_message = """You are a helpful AI assistant. You have access to web search and calculator tools.

    Use the web_search tool when:
    - Asked about current events, news, or recent information
    - Need to find specific facts or data
    - Asked about real-time information (weather, stock prices, etc.)

    Use the calculator tool when:
    - Asked to perform mathematical calculations
    - Need to solve math problems

    For general knowledge questions that don't require real-time data, answer directly.
    Be helpful and conversational in your responses."""

    # Prepare messages with system prompt
    all_messages = [{"role": "system", "content": system_message}] + messages

    # Get response from LLM
    response = llm_with_tools.invoke(all_messages)

    return {"messages": [response]}

def should_continue(state: AgentState) -> Literal["tools", "end"]:
    """
    Determine whether to continue to tools or end the conversation
    """
    print("Judge node invoked")
    messages = state.get("messages", [])
    last_message = messages[-1]

    # If the last message has tool calls, go to tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    # Otherwise, end the conversation
    else:
        return "end"

def answer_node(state: AgentState) -> AgentState:
    print("Answer node invoked")
    user_q = next((m.content for m in reversed(state.get("messages", []))
                   if isinstance(m, HumanMessage)), "")

    ctx_parts = []
    if state.get("rag"):
        ctx_parts.append("Knowledge Base Information:\n" + state["rag"])
    if state.get("web"):
        ctx_parts.append("Web Search Results:\n" + state["web"])

    context = "\n\n".join(ctx_parts) if ctx_parts else "No external context available."

    prompt = f"""Please answer the user's question using the provided context.

Question: {user_q}

Context:
{context}

Provide a helpful, accurate, and concise response based on the available information."""

    ans = answer_llm.invoke([HumanMessage(content=prompt)]).content

    return {
        **state,
        "messages": state["messages"] + [AIMessage(content=ans)]
    }

def router_node(state: AgentState) -> AgentState:
    query = next((m.content for m in reversed(state["messages"])
                  if isinstance(m, HumanMessage)), "")

    # Use structured output properly - pass messages directly
    messages = [
        ("system", (
            "You are a router that decides how to handle user queries:\n"
            "- Use 'end' for pure greetings/small-talk (also provide a 'reply')\n"
            "- Use 'rag' when knowledge base lookup is needed\n"
            "- Use 'answer' when you can answer directly without external info"
        )),
        ("user", query)
    ]

    result: RouteDecision = router_llm.invoke(messages)

    out = {"messages": state["messages"], "route": result.route}
    if result.route == "end":
        out["messages"] = state["messages"] + [AIMessage(content=result.reply or "Hello!")]
    return out

#For uploading files

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, UnstructuredHTMLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
from typing import List
from langchain_core.documents import Document
import os

# Configure text splitter and persistent Chroma client
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, length_function=len)
client = chromadb.PersistentClient(path="/content/chroma_db")
vectorstore = client.get_or_create_collection("documents")


def load_and_split_document(file_path: str) -> List[Document]:
    """
    Load a document from disk and split it into chunks.
    Supported formats: PDF, DOCX, HTML.
    """
    if file_path.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith('.docx'):
        loader = Docx2txtLoader(file_path)
    elif file_path.endswith('.html'):
        loader = UnstructuredHTMLLoader(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_path}")

    documents = loader.load()
    return text_splitter.split_documents(documents)


def index_document_to_chroma(file_path: str, file_id: int) -> bool:
    """
    Index a document by splitting into chunks, assigning unique IDs,
    and storing metadata for each chunk for easy deletion later.
    """
    try:
        splits = load_and_split_document(file_path)
        print(f"working with file {file_id}")

        # Prepare data for Chroma
        documents = [split.page_content for split in splits]
        metadatas = []
        ids = []
        for i, split in enumerate(splits):
            # Unique chunk ID: combine file_id with chunk index
            unique_id = f"{file_id}_{i}"
            ids.append(unique_id)
            # Attach file_id and optional source path to metadata
            md = split.metadata.copy()
            md['file_id'] = str(file_id)
            md['source'] = os.path.basename(file_path)
            metadatas.append(md)

        # Upsert chunks into the collection
        vectorstore.upsert(
            documents=documents,
            ids=ids,
            metadatas=metadatas
        )
        #vectorstore.persist()
        print(f"file {file_id} uploaded")
        return True

    except Exception as e:
        print(f"Error indexing document: {e}")
        return False

def get_relevant_documents(query, k=2):
    results = vectorstore.query(
        query_texts=[query],
        n_results=k
    )
    return results['documents'][0][0]+"\n\n"+results['documents'][0][1]

# index_document_to_chroma('/content/Company_ GreenFields BioTech.docx',1)
# index_document_to_chroma('/content/Company_ QuantumNext Systems.docx',2)
# index_document_to_chroma('/content/Company_ TechWave Innovations.docx',3)
# index_document_to_chroma('/content/GreenGrow Innovations_ Company History.docx',4)
# index_document_to_chroma("/content/GreenGrow's EcoHarvest System_ A Revolution in Farming.pdf",5)

#Tools

In [ ]:
from langchain_tavily import TavilySearch
from langchain_core.tools import tool

tavily = TavilySearch(max_results=3, topic="general")
@tool
def web_search_tool(query: str) -> str:
    """Up-to-date web info via Tavily"""
    print("Web search tool invoked")
    try:
        result = tavily.invoke({"query": query})

        # Extract and format the results from Tavily response
        if isinstance(result, dict) and 'results' in result:
            formatted_results = []
            for item in result['results']:
                title = item.get('title', 'No title')
                content = item.get('content', 'No content')
                url = item.get('url', '')
                formatted_results.append(f"Title: {title}\nContent: {content}\nURL: {url}")

            return "\n\n".join(formatted_results) if formatted_results else "No results found"
        else:
            return str(result)
    except Exception as e:
        return f"WEB_ERROR::{e}"

@tool
def rag_search_tool(query: str) -> str:
    """Top-3 chunks from KB (empty string if none)"""
    print("RAG search tool invoked")
    try:
        return get_relevant_documents(query, k=2)
    except Exception as e:
        return f"RAG_ERROR::{e}"

@tool
def calculator(expression: str) -> str:
    """Calculate mathematical expressions. Use this for any math calculations."""
    print("Calculator tool invoked")
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

#Building graph

In [ ]:
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

def router_node(state: AgentState) -> AgentState:
    query = next((m.content for m in reversed(state["messages"])
                  if isinstance(m, HumanMessage)), "")

    # Use structured output properly - pass messages directly
    messages = [
        ("system", (
            "You are a router that decides how to handle user queries:\n"
            "- Use 'end' for pure greetings/small-talk (also provide a 'reply')\n"
            "- Use 'rag' when knowledge base lookup is needed\n"
            "- Use 'answer' when you can answer directly without external info"
        )),
        ("user", query)
    ]

    result: RouteDecision = router_llm.invoke(messages)

    out = {"messages": state["messages"], "route": result.route}
    if result.route == "end":
        out["messages"] = state["messages"] + [AIMessage(content=result.reply or "Hello!")]
    return out

tools = [calculator, web_search_tool]
llm = ChatGroq(model="llama3-70b-8192", temperature=0.7)
llm_with_tools = llm.bind_tools(tools)
judge_llm  = ChatGroq(model="llama3-70b-8192", temperature=0)\
             .with_structured_output(RagJudge)
answer_llm = ChatGroq(model="llama3-70b-8192", temperature=0.7)
router_llm = ChatGroq(model="llama3-70b-8192", temperature=0)\
             .with_structured_output(RouteDecision)

g = StateGraph(AgentState)
g.add_node("router", router_node)
g.add_node("rag_lookup", rag_node)
g.add_node("web_search", web_node)
g.add_node("answer", answer_node)

g.set_entry_point("router")
g.add_conditional_edges("router", from_router,
                        {"rag": "rag_lookup", "answer": "answer", "end": END})
g.add_conditional_edges("rag_lookup", after_rag,
                        {"answer": "answer", "web": "web_search"})
g.add_edge("web_search",  "answer")
g.add_edge("answer", END)

agent = g.compile(checkpointer=MemorySaver())

#For final run

In [ ]:
if __name__ == "__main__":
    config = {"configurable": {"thread_id": "thread-12"}}
    print("RAG Agent CLI (type 'quit' or 'exit' to stop)")
    print("-" * 50)

    while True:
        q = input("\nYou: ").strip()
        if q.lower() in {"quit", "exit"}:
            break

        try:
            result = agent.invoke(
                {"messages": [HumanMessage(content=q)]},
                config=config
            )

            # Get the last AI message
            last_message = next((m for m in reversed(result["messages"])
                               if isinstance(m, AIMessage)), None)

            if last_message:
                print(f"Agent: {last_message.content}")
            else:
                print("Agent: No response generated")

        except Exception as e:
            print(f"Error: {e}")

    print("\nGoodbye!")